In [38]:
import os
import json
import numpy as np
import tensorflow as tf

In [39]:
# List that holds all the grids to be used as inputs for training
x_train_grid = []
# List that holds all the positions to be used as inputs for training
x_train_position = []
# List that holds all the distances to be used as inputs for training
x_train_distance = []
# List that holds all the directions to be used as outputs for training
y_train = []

# List that holds all the grids to be used as inputs for testing
x_test_grid = []
# List that holds all the positions to be used as inputs for testing
x_test_position = []
# List that holds all the distances to be used as inputs for testing
x_test_distance = []
# List that holds all the directions to be used as outputs for testing
y_test = []

In [40]:
# Name of directory for grids
directory_name = './data/small_grids/'

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    data = json.load(f)

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        x_train_grid.append(i['gridworld'])
        x_train_position.append(i['position'])
        x_train_distance.append(i['distance'])
        y_train.append(i['direction'])
    
    # Close file socket
    f.close()

In [41]:
# Name of directory for grids
directory_name = './data/small_grids_test/'

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    data = json.load(f)

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        x_test_grid.append(i['gridworld'])
        x_test_position.append(i['position'])
        x_test_distance.append(i['distance'])
        y_test.append(i['direction'])
    
    # Close file socket
    f.close()

In [42]:
print(x_train_position)
print(y_train)

[[0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [5, 0], [6, 0], [6, 0], [6, 1], [7, 1], [8, 1], [8, 1], [7, 1], [7, 2], [7, 3], [8, 3], [8, 3], [8, 4], [9, 4], [10, 4], [11, 4], [11, 4], [11, 5], [11, 5], [11, 6], [12, 6], [13, 6], [14, 6], [15, 6], [16, 6], [17, 6], [18, 6], [18, 6], [18, 7], [19, 7], [19, 7], [19, 8], [20, 8], [21, 8], [22, 8], [23, 8], [24, 8], [25, 8], [25, 8], [25, 9], [26, 9], [27, 9], [28, 9], [29, 9], [29, 10], [29, 11], [29, 11], [29, 10], [28, 10], [27, 10], [27, 10], [26, 10], [26, 11], [26, 12], [26, 12], [25, 12], [25, 13], [25, 13], [24, 13], [24, 14], [24, 14], [23, 14], [23, 15], [23, 16], [23, 16], [23, 17], [24, 17], [24, 17], [23, 17], [23, 18], [23, 19], [23, 19], [23, 20], [24, 20], [25, 20], [26, 20], [26, 20], [26, 21], [27, 21], [28, 21], [29, 21], [29, 21], [28, 21], [28, 22], [28, 23], [29, 23], [29, 24], [29, 24], [28, 24], [28, 25], [28, 26], [28, 26], [28, 27], [29, 27], [29, 27], [28, 27], [27, 27], [27, 28], [27, 29], [28, 29], [0, 0], [1, 0], [

In [43]:
# Reshape the data
train_in_grid = np.reshape( x_train_grid, (-1, 30, 30) ) / 2
train_in_position = np.reshape( x_train_position, (-1, 2, 1) ) / 30
train_in_distance = np.reshape( x_train_distance, (-1,1,1) ) / 60
train_out = tf.keras.utils.to_categorical( y_train, 4 )

test_in_grid = np.reshape( x_test_grid, (-1, 30, 30) ) / 2
test_in_position = np.reshape( x_test_position, (-1, 2, 1) ) / 30
test_in_distance = np.reshape( x_test_distance, (-1, 1, 1) ) / 60
test_out = tf.keras.utils.to_categorical( y_test, 4 )

In [44]:
# Flatten the grid input
grid_input = tf.keras.layers.Input( shape = (30,30) )
flatten_grid = tf.keras.layers.Flatten()( grid_input )

In [45]:
# Flatten the position input
position_input = tf.keras.layers.Input( shape = (2,1) )
flatten_position = tf.keras.layers.Flatten()( position_input )

In [46]:
# flatten the distance inputs
distance_input = tf.keras.layers.Input( shape = (1,1) )
flatten_distance = tf.keras.layers.Flatten()( distance_input )

In [47]:
# Concatenate the grid and position into one vector which will be passed to neural network as input
final_input = tf.keras.layers.Concatenate()([flatten_grid, flatten_position, flatten_distance])

In [48]:
# Create layers for Neural Network
dense_1 = tf.keras.layers.Dense( units = 100, activation = tf.nn.relu )( final_input )
dense_2 = tf.keras.layers.Dense( units = 50, activation = tf.nn.relu )( dense_1 )
dense_3 = tf.keras.layers.Dense( units = 10, activation = tf.nn.relu )( dense_2 )
logits = tf.keras.layers.Dense( units = 4, activation = None )( dense_3 )
probabilities = tf.keras.layers.Softmax()( logits )

In [49]:
# Compile the neural network to use stochastic gradient descent as the optimizer and categorical_crossentropy as loss function
model = tf.keras.Model( inputs = [grid_input, position_input, distance_input], outputs = probabilities )
model.compile( optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [50]:
def generate_confusion_matrix( data, labels ):
    mat = [ [ 0 for i in range(4) ] for j in range(4) ]
    
    predictions = np.argmax( model.predict( data ), axis = 1 )
    
    for i in range( data[0].shape[0] ):
        mat[ labels[i] ][ predictions[i] ] += 1
    
    for i in range(4):
        print( "\t".join( [ str(c) for c in mat[i] ] ) )

In [51]:
# Test out before training
generate_confusion_matrix( [test_in_grid, test_in_position, test_in_distance], y_test )

0	0	2	0
0	0	14	0
0	0	40	0
0	0	54	0


In [52]:
# Train the model
history = model.fit( [train_in_grid, train_in_position, train_in_distance], train_out, epochs = 20 )

Epoch 1/20
22/22 [==============================] - 3s 20ms/step - loss: 1.2017 - accuracy: 0.4271
Epoch 2/20
22/22 [==============================] - 0s 16ms/step - loss: 1.1333 - accuracy: 0.4845
Epoch 3/20
22/22 [==============================] - 0s 12ms/step - loss: 1.1135 - accuracy: 0.4934
Epoch 4/20
22/22 [==============================] - 0s 18ms/step - loss: 1.1189 - accuracy: 0.4860
Epoch 5/20
22/22 [==============================] - 0s 15ms/step - loss: 1.1027 - accuracy: 0.5066
Epoch 6/20
22/22 [==============================] - 0s 11ms/step - loss: 1.1060 - accuracy: 0.5007
Epoch 7/20
22/22 [==============================] - 0s 16ms/step - loss: 1.0962 - accuracy: 0.4875
Epoch 8/20
22/22 [==============================] - 0s 10ms/step - loss: 1.1023 - accuracy: 0.5022
Epoch 9/20
22/22 [==============================] - 0s 13ms/step - loss: 1.0947 - accuracy: 0.5022
Epoch 10/20
22/22 [==============================] - 0s 10ms/step - loss: 1.0838 - accuracy: 0.5184
Epoch 11/

In [53]:
# Test out after training
generate_confusion_matrix( [test_in_grid, test_in_position, test_in_distance], y_test )

0	0	0	2
0	0	0	14
0	0	0	40
0	0	0	54
